<center><h1>Colony Initializer</h1></center>

Method to randomly initialize a colony with $n$ ants and objetive function values.

 > The food sources will be represented by the solution of the chosen Power System configuration

In [ ]:
from acopoweropt import colony, system

PSystem = system.PowerSystem(name='s10')

Colony = colony.Colony(n_ants=5, phr_evp_rate={'worst': 0.4, 'mean': 0.25, 'best': 0.05}, power_system_name='s10')
Colony.initialize()

print(Colony.initial_paths)

In [ ]:
n_ants = 5

def initialize(n_ants: int):
    """Initialize colony
    
    Initializes colony by seting the ants towards random paths. Although not ideal
    each random path will make use of PowerSystem.sample_operation() and the path
    distance will be calculated using PowerSystem.solve(operation).
    
    Later improvements should aim to decouple the PowerSystem from within the colony
    initialization method.
    
    """
    
    l = []
    for ant in range(1, n_ants + 1):
        option = PSystem.sample_operation()
        result = PSystem.solve(operation=option)
        distance = trail_result.get('Ft')
        status = trail_result.get('status')

        l.append({'ant': ant, 
                  'path': option.opz.to_list(), 
                  'status': status,
                  'distance':distance,
                  'tau': 1/distance})
        
    return pd.DataFrame(l).set_index('ant') 

## Iteractively update pheromone

In [1]:
import pandas as pd
import random

from acopoweropt import colony, system

PSystem = system.PowerSystem(name='s15')
Colony = colony.PowerColony(n_ants=5,
                            pheromone_evp_rate={'worst': 0.4, 'mean': 0.25, 'best': 0.05},
                            power_system=PSystem)

In [2]:
Colony.pheromone.copy()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
opz,,,,,,,,,,,,,,,
1,0.1526,0.0304,0.1526,0.1526,0.0609,0.0303,0.1526,0.1526,0.1526,0.1526,0.1526,0.0303,0.1526,0.1526,0.1526
2,0.0000,0.0608,0.0000,0.0000,0.0610,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0614,0.0000,0.0000,0.0000
3,0.0000,0.0307,0.0000,0.0000,0.0000,0.0609,0.0000,0.0000,0.0000,0.0000,0.0000,0.0609,0.0000,0.0000,0.0000
4,0.0000,0.0307,0.0000,0.0000,0.0307,0.0614,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [2]:
Colony.evaporate_pheromone(paths=Colony.paths[0], power_system=PSystem)

In [3]:
Colony.pheromone

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
opz,,,,,,,,,,,,,,,
1,0.118388,0.018120,0.118388,0.118388,0.018120,0.000000,0.118388,0.118388,0.118388,0.118388,0.118388,0.021924,0.118388,0.118388,0.118388
2,0.000000,0.023025,0.000000,0.000000,0.000000,0.021924,0.000000,0.000000,0.000000,0.000000,0.000000,0.029165,0.000000,0.000000,0.000000
3,0.000000,0.023025,0.000000,0.000000,0.029165,0.055413,0.000000,0.000000,0.000000,0.000000,0.000000,0.034537,0.000000,0.000000,0.000000
4,0.000000,0.055413,0.000000,0.000000,0.038855,0.023025,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [2]:
# colony.evaporate()

paths = Colony.paths[0].copy()
pheromone = Colony.pheromone.copy()
pheromone_evp_rate={'worst': 0.4, 'mean': 0.25, 'best': 0.05}

best_path = paths[paths.distance == paths.distance.min()].iloc[0].path
worst_path = paths[paths.distance == paths.distance.max()].iloc[0].path

In [12]:
def evaporate_pheromone(paths: pd.DataFrame):

    for ant in paths.itertuples():
        distance = ant.distance

        for i, opz in enumerate(ant.path):
            tgu = i+1 # TGUs are indexed from 1
            opz = int(opz)

            if PSystem.operative_zones[tgu] == 1:
                evaporation = (1 - pheromone_evp_rate['best'])
            else:
                if opz == best_path[i]:
                    evaporation = (1 - pheromone_evp_rate['best'])
                elif opz == worst_path[i]:
                    evaporation = (1 - pheromone_evp_rate['worst'])
                else:
                    evaporation = (1 - pheromone_evp_rate['mean'])

            pheromone.at[opz, tgu] = evaporation * pheromone.at[opz, tgu]

In [13]:
pheromone

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
opz,,,,,,,,,,,,,,,
1,0.118388,0.000000,0.118388,0.118388,0.023025,0.021924,0.118388,0.118388,0.118388,0.118388,0.118388,0.022875,0.118388,0.118388,0.118388
2,0.000000,0.018240,0.000000,0.000000,0.021924,0.023025,0.000000,0.000000,0.000000,0.000000,0.000000,0.078707,0.000000,0.000000,0.000000
3,0.000000,0.029165,0.000000,0.000000,0.055413,0.023025,0.000000,0.000000,0.000000,0.000000,0.000000,0.023025,0.000000,0.000000,0.000000
4,0.000000,0.038770,0.000000,0.000000,0.000000,0.029165,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
pheromone_evp_rate={'worst': 0.4, 'mean': 0.25, 'best': 0.05}

for ant in paths.itertuples():
    distance = ant.distance
    pheromone = round(1000 / distance, 4)

    for i, opz in enumerate(ant.path):
        tgu = i+1 # TGUs are indexed from 1
        opz = int(opz)
        
        if PSystem.operative_zones[tgu] == 1:
            pheromone.at[opz, tgu] = (1 - pheromone_evp_rate['best']) * Colony.pheromone
            
            pheromone.at[opz, tgu] = pheromone.at[opz,]
        
        tgu = i + 1
        opz = int(opz)
        pheromone = round(1000 / distance, 4)

        pheromone.at[opz, tgu] = pheromone.at[opz, tgu] + pheromone

return self.pheromone

In [1]:
paths = Colony.paths[0].copy()
pheromone = Colony.pheromone.copy()
pheromone_evp_rate={'worst': 0.4, 'mean': 0.25, 'best': 0.05}


best_path = paths[paths.distance == paths.distance.min()].path
worst_path = paths[paths.distance == paths.distance.max()].path

for ant in paths.itertuples():
    distance = ant.distance

    for i, opz in enumerate(ant.path):
        tgu = i+1 # TGUs are indexed from 1
        opz = int(opz)
        
        if PSystem.operative_zones[tgu] == 1:
            evaporation = (1 - pheromone_evp_rate['best'])
        else:
            if opz == best_path[i]:
                evaporation = (1 - pheromone_evp_rate['best'])
            elif opz == worst_path[i]:
                evaporation = (1 - pheromone_evp_rate['worst'])
            else:
                evaporation = (1 - pheromone_evp_rate['mean'])
            
        print(ant, tgu, opz, evaporation)
        #pheromone.at[opz, tgu] = evaporation * pheromone.at[opz, tgu]

NameError: name 'Colony' is not defined